# Practica 3 - Multiclasificadores y selección de variables

## Minería de Datos 2017/2018

* [**Hernán Indíbil de La Cruz Calvo**](https://github.com/Mowstyl)
* [**Alejandro Martín Simón Sánchez**](https://github.com/elssbbboy/)

## Introducción

Esta práctica se divide en dos partes:

**Multiclasificadores:** estudiaremos la API de scikit para los modelos de tipo ensemble y veremos como entrenar, seleccionar y utilizar estos modelos.

**Selección de variables:** veremos los distintos algoritmos de selección de variables disponibles en scikit y como aplicarlos.

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
seed=6470
np.random.seed(seed)

In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_diabetes
dss = {}
dss['wisconsin'] = load_breast_cancer()
dss['pima'] = load_diabetes()
dir(dss['wisconsin'])

['DESCR', 'data', 'feature_names', 'target', 'target_names']

Como podemos ver, en dss['wisconsin'] tenemos los siguientes atributos:

* DESCR: descripción del dataset, con numero de casos, codificación de los valores perdidos, atributos, información de los mismos además de estadísticos y información adicional sobre la obtención de los datos y su origen.
* data: array de numpy con los valores de cada atributo para cada caso. En prácticas anteriores sería similar al train_atts o test_atts (aún no hemos hecho holdout), cambiando que no se utiliza pandas y no tenemos los nombres de las variables predictoras.
* feature_names: nombres de las variables categóricas, usado como columna en pandas anteriormente. De esta forma, dfs['wisconsin'].data[n][m] indica un valor de la variable dfs['wisconsin'].feature_names[m].
* target: array de numpy con los valores de la clase para cada instancia. No se trabaja con strings, sino que ha sido binarizada para trabajar con 0 y 1 (más eficiente que trabajar con strings).
* target_names: nombres de los distintos valores que toma la variable categórica original. De esta forma si tenemos un 0 en target sabemos que tenemos que se clasifica como target_names[0], en este caso 'malignant'.


In [5]:
dir(dss['pima'])

['DESCR', 'data', 'feature_names', 'target']

Como podemos ver, en dss['pima'] tenemos algunas diferencias con el dataset wisconsin:

* En target ya no tenemos 0 y 1, ya que la variable clase no es categórica, sino una medida cuantitativa de la progresión de la enfermedad un año después de la baseline.
* El atributo target_names no tiene sentido ya que no existe ninguna correspondencia como en wisconsin. La variable clase no es categórica.


In [6]:
from sklearn.model_selection import train_test_split
dssh = {}

for ds in dss:
    strat = None
    if ('target_names' in dir(dss[ds])): # Solo tiene sentido estratificar si la clase es categórica
        strat = dss[ds].target
    
    dssh[ds] = {'train': {}, 'test': {}}
    dssh[ds]['train']['atts'], dssh[ds]['test']['atts'], dssh[ds]['train']['label'], dssh[ds]['test']['label'] = train_test_split(
        dss[ds].data,
        dss[ds].target,
        random_state = seed,
        stratify = strat)

La siguiente función lleva a cabo un muestreo de variables predictoras. Los parámetros que se le pueden pasar son:
* atts: array con los casos, preferiblemente de numpy. train_atts.
* label: array con la variable clase asociada a cada caso. train_label.
* feature_names: nombres de las variables predictoras. Opcional.
* n_samples: número de muestras a generar. Opcional.
* best_features: parámetro que indica si se tomarán las mejores variables para cada muestra. Si hay reemplazo no tiene sentido, ya que las n muestras que genere serán idénticas. Opcional.
* replace: parámetro que indica si el muestreo se realiza con reemplazo. Opcional.
* random_state: parámetro que indica la semilla para generar números aleatorios, solo utilizada con best_features=False. Opcional.
* k: número de variables en cada muestra en caso de haber reemplazo y que best_features=False. Opcional.

Devuelve una tripla con:
* Lista de muestras(solo los casos/atts).
* Lista de muestras(solo la clase/label).
* Lista de listas con nombres de variables predictoras usadas en cada muestra. Si no se especificó feature_names devuelve None.

In [7]:
import random
from sklearn.feature_selection import SelectKBest
def sampleFeatures(atts, label, feature_names=None, n_samples=4, best_features=True, replace=False,
                   random_state=None, k=10):
    if (random_state is not None and not best_features):
        random.seed(random_state)
    kaux = int(atts.shape[1]/n_samples)
    ks = [kaux for i in range(0, n_samples)]
    if (kaux * n_samples < atts.shape[1]):
        ks[-1] += 1
    
    auxatts = atts[:,:]
    fnames = None
    if (feature_names is not None):
        auxfname = feature_names[:]
        fnames = []
    
    sample = []
    labels = []
    
    for s in ks:
        if (best_features):
            sel = SelectKBest(k=s)
            sel.fit(auxatts, label)
            mask = sel.get_support()
        else:
            cols = s
            if (replace):
                cols = k
            samp = random.sample(range(0, auxatts.shape[1]), cols)
            mask = [i in samp for i in range(0, auxatts.shape[1])]
        sample.append(auxatts[:,mask])
        labels.append(label)
        if (feature_names is not None):
            fnames.append(auxfname[mask])
            if(not replace):
                auxfname = auxfname[np.invert(mask)]
        if(not replace):
            auxatts = auxatts[:,np.invert(mask)]
    return np.array(sample), np.array(labels), np.array(fnames)

La siguiente función lleva a cabo el muestreo, sobre variables o sobre casos, con o sin reemplazo. Los parámetros que se le pueden pasar son:
* atts: array con los casos, preferiblemente de numpy. train_atts.
* label: array con la variable clase asociada a cada caso. train_label.
* feature_names: nombres de las variables predictoras. Opcional.
* n_samples: número de muestras a generar. Opcional.
* sample_features: indica si el muestreo se realiza sobre los casos (False) o sobre las variables predictoras (True)
* best_features: parámetro que indica si se tomarán las mejores variables para cada muestra. Si hay reemplazo no tiene sentido, ya que las n muestras que genere serán idénticas. Solo se toma si sample_features=True. Opcional.
* replace: parámetro que indica si el muestreo se realiza con reemplazo (sobre las variables predictoras o sobre los casos según el valor de sample_features). Opcional.
* random_state: parámetro que indica la semilla para generar números aleatorios, solo utilizada con best_features=False. Opcional.
* k: número de variables en cada muestra en caso de haber reemplazo, sample_features=True y best_features=False. Opcional.

Devuelve una tripla con:
* Lista de muestras(solo los casos/atts).
* Lista de muestras(solo la clase/label).
* Lista de listas con nombres de variables predictoras usadas en cada muestra. Si no se especificó feature_names devuelve None.

In [8]:
from sklearn.utils import resample
def ourSample(atts, label, feature_names=None, random_state=None, n_samples=2,
              replace=True, sample_features=False, best_features=True, k=10):
    if (sample_features):
        return sampleFeatures(atts, label, feature_names=feature_names, random_state=random_state,
                              best_features=best_features, n_samples=n_samples, replace=replace, k=k)
    elif(replace):
        sample = []
        labels = []
        fnames = None
        if (feature_names is not None):
            fnames = []
        for i in range(0, n_samples):
            X, y = resample(atts, label, random_state=random_state, n_samples=n_samples, replace=True)
            sample.append(X)
            labels.append(y)
            if (feature_names is not None):
                fnames.append(feature_names)
        return np.array(sample), np.array(labels), np.array(fnames)
    else:
        if (random_state is not None):
            random.seed(random_state)
        ncases = int(atts.shape[0]/n_samples)
        ncss = [ncases for i in range(0, n_samples)]
        if (ncases * n_samples < atts.shape[0]):
            ncss[-1] += 1
        auxatts = atts[:,:]
        auxlabels = label[:]
        sample = []
        labels = []
        fnames = None
        if (feature_names is not None):
            fnames = []
        for nc in ncss:
            samp = random.sample(range(0, auxatts.shape[0]), nc)
            mask = [i in samp for i in range(0, auxatts.shape[0])]
            sample.append(auxatts[mask])
            labels.append(auxlabels[mask])
            if (feature_names is not None):
                fnames.append(feature_names)
            auxatts = auxatts[np.invert(mask)]
            auxlabels = auxlabels[np.invert(mask)]
        return np.array(sample), np.array(labels), np.array(fnames)

Ejemplo de uso:
Vamos a generar una lista de 10 muestras en las que muestreemos las variables predictoras. Cada muestra tendrá 10 variables predictoras escogidas aleatoriamente

In [9]:
a, l, n = ourSample(dssh['wisconsin']['train']['atts'],
                    dssh['wisconsin']['train']['label'],
                    feature_names=dss['wisconsin'].feature_names, n_samples=10,
                    replace=True, best_features=False, sample_features=True)

Así, en a[0] tenemos nuestra primera muestra, donde a[0][0] sería el primer caso

In [10]:
a[0][0]

array([  1.04900000e+01,   6.74100000e+01,   9.98900000e-02,
         8.57800000e-02,   1.53400000e+00,   2.88000000e-02,
         7.42200000e+01,   1.21900000e-01,   1.48600000e-01,
         2.82600000e-01])

en l[0][0] se tendrá el valor de la clase para el primer caso.

In [11]:
l[0][0]

1

En n[0] se tendrá el nombre de las variables predictoras usadas en la primera muestra.

In [12]:
n[0]

array(['mean radius', 'mean perimeter', 'mean smoothness',
       'mean compactness', 'texture error', 'concavity error',
       'worst perimeter', 'worst smoothness', 'worst compactness',
       'worst symmetry'],
      dtype='<U23')

Una vez listo el método para realizar el muestreo, podemos proceder a diseñar el ensemble.
Tomará como parámetros los necesarios para ajustar el algoritmo del muestreo además de el algoritmo que se usará para el ensemble. Dicho algoritmo vendrá dado con una clase cuyas instancias tengan implementados los métodos set_params, fit y predict. Además, se podrá dar otro parámetro al ensemble llamado params que se podrá usar para indicar los parámetros que se pasan a todas las instancias del algoritmo al crear el ensemble.
Cabe destacar que en lugar de n_samples tendremos n_models, que indicará tanto el número de muestras como el número de estimadores generados.
El ensemble implementará a su vez los métodos fit y predict.

In [13]:
from scipy import stats
class EnsembleHome:
    def __init__(self, est, random_state=None, n_models=10, replace=True,
                 sample_features=False, best_features=True, k=10, params=None):
        self.base = est
        self.rstate = random_state
        self.nmodels = n_models
        self.replace = replace
        self.sfeatures = sample_features
        self.bfeatures = best_features
        self.k = k
        self.params = params
        
        self.ests = []
        for i in range(0, n_models):
            self.ests.append(est())
            if (params is not None):
                self.ests[i].set_params(**params)
        
    def fit(self, atts, label, feature_names=None):
        self.fnames = feature_names
        satts, slabels, sfname = ourSample(atts, label, feature_names=feature_names,
                                           n_samples=self.nmodels, replace=self.replace, random_state=self.rstate,
                                           best_features=self.bfeatures, sample_features=self.sfeatures)
        self.satts = satts
        self.slabels = slabels
        self.sfname = sfname
        for i in range(0, self.nmodels):
            self.ests[i].fit(self.satts[i], self.slabels[i])
    
    def predict(self, atts):
        predictions = []
        for i in range(len(self.ests)):
            mask = [self.fnames[j] in self.sfname[i] for j in range(len(self.fnames))]
            paux = self.ests[i].predict(atts[:,mask])
            predictions.append(paux)
        return stats.mode(predictions)[0][0]
        #return np.stats.mode(predictions)

Muestreo con reemplazo.

In [14]:
from sklearn import tree
ensemble = EnsembleHome(tree.DecisionTreeClassifier, random_state=seed, n_models=10)

In [15]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [16]:
p = {}
p['Con reemplazo, sin muestreo de atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

A continuación se realiza el muestreo sin reemplazo.

In [17]:
ensemble = EnsembleHome(tree.DecisionTreeClassifier, replace = False, random_state=seed, n_models=10)

In [18]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [19]:
p['Sin reemplazo, sin muestreo de atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

In [20]:
ensemble = EnsembleHome(tree.DecisionTreeClassifier, replace = False, sample_features=True, best_features=False, random_state=seed, n_models=10)

In [21]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [22]:
p['Sin reemplazo, con muestreo de atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

In [23]:
ensemble = EnsembleHome(tree.DecisionTreeClassifier, replace = True, sample_features=True, best_features=False, random_state=seed, n_models=10)

In [24]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [25]:
p['Con reemplazo, con muestreo de atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

In [26]:
ensemble = EnsembleHome(tree.DecisionTreeClassifier, replace = False, sample_features=True, best_features=True, random_state=seed, n_models=10)

In [27]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [28]:
p['Conjuntos con los mejores atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

In [29]:
import sklearn.metrics as metrics
# Metodo que pasado test_label y un diccionario con predicciones indizadas
#   por el algoritmo devuelve una tabla con las metricas utilizadas

def metricTable(test_label, predictions, pos_label):
    algL = []
    accuracyL = []
    recallL = []
    precisionL = []
    f1scoreL = []
    aucL = []

    for alg, prediction in predictions.items():
        algL.append(alg)
        accuracyL.append(metrics.accuracy_score(test_label, prediction))
        recallL.append(metrics.recall_score(test_label, prediction, pos_label=pos_label))
        precisionL.append(metrics.precision_score(test_label, prediction, pos_label=pos_label))
        f1scoreL.append(metrics.f1_score(test_label, prediction, pos_label=pos_label))
        aucL.append(metrics.roc_auc_score(y_true=pd.get_dummies(test_label), y_score=pd.get_dummies(prediction)))

    table = [('Algorithm', algL),
             ('Accuracy', accuracyL),
             ('Recall', recallL),
             ('Precision', precisionL),
             ('F1 Score', f1scoreL),
             ('ROC AUC', aucL)
             ]

    return pd.DataFrame.from_items(table)

def cMatrix(matrix, pos_label, neg_label):
    rowL = [pos_label, neg_label]
    tColL = [matrix[1,1], matrix[0,1]]
    fColL = [matrix[1,0], matrix[0,0]]
    table = [('Actual \ Pred', rowL),
             (pos_label, tColL),
             (neg_label, fColL)]
    return pd.DataFrame.from_items(table)

In [30]:
metricTable(dssh['wisconsin']['test']['label'], p, 0)

,Algorithm,Accuracy,Recall,Precision,F1 Score,ROC AUC
0,"Con reemplazo, sin muestreo de atributos",0.895105,0.830189,0.880000,0.854369,0.881761
1,"Sin reemplazo, sin muestreo de atributos",0.916084,0.849057,0.918367,0.882353,0.902306
2,"Sin reemplazo, con muestreo de atributos",0.937063,0.867925,0.958333,0.910891,0.922851
3,"Con reemplazo, con muestreo de atributos",0.937063,0.886792,0.940000,0.912621,0.926730
4,Conjuntos con los mejores atributos,0.902098,0.830189,0.897959,0.862745,0.887317
